In [38]:
import numpy as np
import cv2
import os
from keras import models, layers, optimizers
from sklearn.model_selection import KFold

In [39]:
spacing = {
    "single_ch": [14.3, 15.3, 16.3, 17.8, 19.3, 21.3, 23.3, 26.3, 28.3, 31.3, 36.3],
    "18GHz": [18, 19, 20, 23, 25, 27, 30, 32, 35, 40],
    "17.6GHz": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "17GHz": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "16.5GHz": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "16GHz": [18, 19, 20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "15.5GHz": [20, 21.5, 23, 25, 27, 30, 32, 35, 40],
    "15GHz": [23, 25, 27, 30, 32, 35, 40],
}

In [40]:
def get_osnr_value(value):
    aux = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
    dict_osnr_norm = {
        18: aux[0],
        19: aux[1],
        20: aux[2],
        21.5: aux[3],
        23: aux[4],
        25: aux[5],
        27: aux[6],
        30: aux[7],
        32: aux[8],
        35: aux[9],
        40: aux[10],
    }
    return dict_osnr_norm[value]

def get_spacing_value(value):
    dict_spacing = {
        "single_ch": 0,
        "18GHz": 1,
        "17.6GHz": 2,
        "17GHz": 3,
        "16.5GHz": 4,
        "16GHz": 5,
        "15.5GHz": 6,
        "15GHz": 7,
    }
    return dict_spacing[value]

In [41]:
dir = "const_gray"
X = []
y = []
f = []

for key, value in spacing.items():
    for val in value:
        for k in range(9):
            # Append the image
            img = cv2.imread(
                os.path.join(dir, key, "{}_dB_sample_{}.png".format(val, k)),
                cv2.IMREAD_GRAYSCALE,
            ).astype("float32")

            # Pre-process the image
            img = cv2.GaussianBlur(img, (5, 5), 0)
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)

            # Append the image
            X.append(img)

            # Append the label
            y.append(get_osnr_value(val + 3.7) if key == "single_ch" else get_osnr_value(val))

            # Append the feature
            f.append(get_spacing_value(key))

X = np.array(X)
f = np.array(f)
y = np.array(y)

In [42]:
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
f = f.reshape(f.shape[0], 1)

In [43]:
n_classes = len(np.unique(y))
input_shape = X[0].shape
feature_shape = f[0].shape

print("Number of Classes: ", n_classes)
print("Input Shape: ", input_shape)
print("Feature Shape: ", feature_shape)

Number of Classes:  2
Input Shape:  (64, 64, 1)
Feature Shape:  (1,)


In [44]:
def create_model(input_shape, feature_shape, mode="classifier", n_classes=2, ch_on=False):
    # Create the Input layer
    input_layer = layers.Input(shape=input_shape, name="input")

    # Create the Convolutional layers
    conv1 = layers.Conv2D(32, (3, 3), activation="relu", name="conv1")(input_layer)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2), name="pool1")(conv1)
    conv2 = layers.Conv2D(16, (3, 3), activation="relu", name="conv2")(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2), name="pool2")(conv2)
    flatten = layers.Flatten(name="flatten")(pool2)

    # Add the feature input
    if ch_on:
        feature = layers.Input(shape=feature_shape, name="feature")
        concat = layers.concatenate([flatten, feature], name="concat")
    else:
        concat = flatten

    # Create the Dense layers
    dense1 = layers.Dense(16, activation="relu", name="dense1")(concat)
    
    # Create the Output layer
    if mode == "classifier":
        output_layer = layers.Dense(n_classes, activation="softmax", name="output")(dense1)
        if ch_on:
            model = models.Model(inputs=[input_layer, feature], outputs=output_layer)
        else:
            model = models.Model(inputs=input_layer, outputs=output_layer)
    elif mode == "regressor":
        output_layer = layers.Dense(1, activation="linear", name="output")(dense1)
        if ch_on:
            model = models.Model(inputs=[input_layer, feature], outputs=output_layer)
        else:
            model = models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [45]:
losses = []
metrics = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    f_train, f_test = f[train_index], f[test_index]

    # Create the model for regression
    # model = create_model(input_shape, feature_shape, mode="regressor", ch_on=False)
    # model.compile(optimizer=optimizers.Adam(), loss="mean_squared_error", metrics=["mean_absolute_error"])

    # Create the model for classification
    model = create_model(input_shape, feature_shape, mode="classifier", n_classes=n_classes, ch_on=True)
    model.compile(optimizer=optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # Train the model
    model.fit([X_train, f_train], y_train, batch_size=32, epochs=100, validation_data=([X_test, f_test], y_test))

    # Evaluate the model
    loss, metric = model.evaluate([X_test, f_test], y_test)

    # losses.append(np.sqrt(loss))
    losses.append(loss)
    metrics.append(metric)

losses = np.array(losses)
metrics = np.array(metrics)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.5767 - loss: 0.6453 - val_accuracy: 0.7534 - val_loss: 0.4471
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8071 - loss: 0.4055 - val_accuracy: 0.7740 - val_loss: 0.3711
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7753 - loss: 0.3991 - val_accuracy: 0.7945 - val_loss: 0.3329
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8176 - loss: 0.3513 - val_accuracy: 0.8219 - val_loss: 0.2991
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8343 - loss: 0.3180 - val_accuracy: 0.8151 - val_loss: 0.3524
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8593 - loss: 0.3013 - val_accuracy: 0.8767 - val_loss: 0.2987
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8690 - loss: 0.2901 - val_accuracy: 0.8904 - val_loss: 0.2191
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8691 - loss: 0.2527 - val_accuracy: 0.

In [46]:
print(metrics)
print("Max accuracy: %0.2f in index %d" % (metrics.max(), metrics.argmax()))
print("%0.2f accuracy with a std of %0.2f" % (metrics.mean(), metrics.std()))

[0.99315071 0.98630136 0.97945207 0.98630136 0.99310344]
Max accuracy: 0.99 in index 0
0.99 accuracy with a std of 0.01


In [47]:
# print(losses)
# print("Min RMSE: %0.2f in index %d" % (np.min(losses), np.argmin(losses)))
# print("%0.2f RMSE with a std of %0.2f" % (np.mean(losses), np.std(losses)))

In [48]:
# print(metrics)
# print("Min MAE: %0.2f in index %d" % (np.min(metrics), np.argmin(metrics)))
# print("%0.2f MAE with a std of %0.2f" % (np.mean(metrics), np.std(metrics)))